In [ ]:
!pip install pymdptoolbox --user

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import mdptoolbox
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

In [2]:
plt.plot([1,2,3],[4,2,5])
plt.show()

In [6]:
lmd0 = 0.013364
lmd1 = 0.333442
lmdM = 1 - lmd0 - lmd1 #0.6531...
mu0 = 0.125
mu1 = 0.25
muM = 0.5

In [7]:
action = ['no maintenance','maintenance']

In [8]:
a0_tm = np.array([[lmdM, lmd1, 0, 0, 0, 0, 0, 0, 0, lmd0], #current state 0 to next state
                  [0, lmdM, lmd1, 0, 0, 0, 0, 0, 0, lmd0], #current state 1 to next state
                  [0, 0, lmdM, lmd1, 0, 0, 0, 0, 0, lmd0], #current state 2 to next state
                  [0, 0, 0, lmdM, 0, 0, 0, 0, lmd1, lmd0], #current state 3 to next state
                  [muM, 0, 0, 0, 1-muM, 0, 0, 0, 0, 0], #current state 4 to next state
                  [muM, 0, 0, 0, 0, 1-muM, 0, 0, 0, 0], #current state 5 to next state
                  [0, muM, 0, 0, 0, 0, 1-muM, 0, 0, 0], #current state 6 to next state
                  [0, 0, muM, 0, 0, 0, 0, 1-muM, 0, 0], #current state 7 to next state
                  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], #current state 8 to next state
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]) #current state 9 to next state


#transition matrix for a = 1 (maintenance steps)
a1_tm = np.array([[0, 0, 0, 0, 1-lmd0, 0, 0, 0, 0, lmd0], #current state 0 to next state
                  [0, 0, 0, 0, 0, 1-lmd0, 0, 0, 0, lmd0], #current state 1 to next state
                  [0, 0, 0, 0, 0, 0, 1-lmd0, 0, 0, lmd0], #current state 2 to next state
                  [0, 0, 0, 0, 0, 0, 0, 1-lmd0, 0, lmd0], #current state 3 to next state
                  [muM, 0, 0, 0, 1-muM, 0, 0, 0, 0, 0], #current state 4 to next state
                  [muM, 0, 0, 0, 0, 1-muM, 0, 0, 0, 0], #current state 5 to next state
                  [0, muM, 0, 0, 0, 0, 1-muM, 0, 0, 0], #current state 6 to next state
                  [0, 0, muM, 0, 0, 0, 0, 1-muM, 0, 0], #current state 7 to next state
                  [mu1, 0, 0, 0, 0, 0, 0, 0, 1-mu1, 0], #current state 8 to next state
                  [mu0, 0, 0, 0, 0, 0, 0, 0, 0, 1-mu0]]) #current state 9 to next state

tm = [a0_tm,a1_tm]

In [9]:
P = np.stack((a0_tm,a1_tm))
P.shape

(2, 10, 10)

In [10]:
trans_main = {0:[4,9],1:[5,9],2:[6,9],3:[7,9],4:[4,0],5:[5,0],6:[6,1],7:[7,2],8:[0],9:[0]}
trans_no_main = {0:[0,1,9],1:[1,2,9],2:[2,3,9],3:[3,8,9],4:[4,0],5:[5,0],6:[6,1],7:[7,2],8:[8],9:[9]}
reward_func = {0:1000,1:900,2:800,3:500,4:-500,5:-500,6:-500,7:-500,8:-3000,9:-1000}
# maintenance_cost = 1000
# reward_func = {0:2000,1:1500,2:1000,3:500,4:-maintenance_cost,5:-maintenance_cost,6:-maintenance_cost,7:-maintenance_cost,8:-3000,9:-2000}

def exp_reward(trans,reward_func):
    reward = []
    
    for row,col in trans.items():
        total = 0
        for j in col:
            total+=reward_func[j]
            
        reward.append(total/len(col))
    return reward

R1 = np.array(exp_reward(trans_main,reward_func))
R1 = np.expand_dims(R1,1)
R0 = np.array(exp_reward(trans_no_main,reward_func))
R0 = np.expand_dims(R0,1)
R = np.hstack((R0,R1))
R

array([[  300.        ,  -750.        ],
       [  233.33333333,  -750.        ],
       [  100.        ,  -750.        ],
       [-1166.66666667,  -750.        ],
       [  250.        ,   250.        ],
       [  250.        ,   250.        ],
       [  200.        ,   200.        ],
       [  150.        ,   150.        ],
       [-3000.        ,  1000.        ],
       [-1000.        ,  1000.        ]])

In [11]:
epsilon = 0.8 #discount factor


In [12]:
#Value Iteration

vi = mdptoolbox.mdp.ValueIteration(P, R, epsilon)
vi.run()

In [13]:
vi.policy

(0, 0, 0, 1, 0, 0, 0, 0, 1, 1)

In [14]:
vi.V

(1031.8835814615386,
 574.9328528534326,
 6.984952861149225,
 -509.71090816181413,
 1104.5687872013461,
 1104.5687872013461,
 716.6017778349459,
 254.63660633399166,
 3015.9113625184646,
 3677.2487109094923)

In [ ]:
vi.iter

In [ ]:
#Policy iteration

PI = mdptoolbox.mdp.PolicyIteration(P, R, epsilon)
#PI.setVerbose()
PI.run()

In [ ]:
PI.policy

In [ ]:
PI.iter

In [ ]:
PI.V

In [ ]:
#Q learning
Ql =  mdptoolbox.mdp.QLearning(P, R, epsilon)
Ql.run()

In [ ]:
Ql.Q

In [ ]:
Ql.policy

In [ ]:
for i in range(len(Ql.policy)):
    print(f"Best action in state {i} is {action[Ql.policy[i]]}")

Plotting Rewards over time following this policy

In [ ]:

pol = vi.policy
rewards = [0]
r = 0
timesteps = 3000
state = 0

for i in range(timesteps-1):
    action = pol[state]
    transition_mat_action = tm[action]
    nxt_state = np.random.choice([i for i in range(10)],1,p=transition_mat_action[state])[0] #transition
    r = reward_func[nxt_state]
    rewards.append(rewards[-1]+r) #cumulative rewards
    state = nxt_state
    
t = [i for i in range(timesteps)]
fig = plt.figure(figsize= (10,10))
plt.plot(t, rewards)
plt.title("Value Iteration")
plt.ylabel('Cumulative Return')
plt.xlabel('Timestep')
plt.show()

Average across 20 timestep

In [ ]:
from machine import Machine
import numpy as np
#Parameters
lmd0 = 0.013364
lmd1 = 0.333442
lmdM = 1 - lmd0 - lmd1 #0.6531...
mu0 = 0.125
mu1 = 0.25
muM = 0.5
maintenance_cost = 750


#transition matrices

#transition matrix for a = 0 (no maintenance)
a0_tm = np.array([[lmdM, lmd1, 0, 0, 0, 0, 0, 0, 0, lmd0], #current state 0 to next state
                  [0, lmdM, lmd1, 0, 0, 0, 0, 0, 0, lmd0], #current state 1 to next state
                  [0, 0, lmdM, lmd1, 0, 0, 0, 0, 0, lmd0], #current state 2 to next state
                  [0, 0, 0, lmdM, 0, 0, 0, 0, lmd1, lmd0], #current state 3 to next state
                  [muM, 0, 0, 0, 1-muM, 0, 0, 0, 0, 0], #current state 4 to next state
                  [muM, 0, 0, 0, 0, 1-muM, 0, 0, 0, 0], #current state 5 to next state
                  [0, muM, 0, 0, 0, 0, 1-muM, 0, 0, 0], #current state 6 to next state
                  [0, 0, muM, 0, 0, 0, 0, 1-muM, 0, 0], #current state 7 to next state
                  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], #current state 8 to next state
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]) #current state 9 to next state


#transition matrix for a = 1 (maintenance steps)
a1_tm = np.array([[0, 0, 0, 0, 1-lmd0, 0, 0, 0, 0, lmd0], #current state 0 to next state
                  [0, 0, 0, 0, 0, 1-lmd0, 0, 0, 0, lmd0], #current state 1 to next state
                  [0, 0, 0, 0, 0, 0, 1-lmd0, 0, 0, lmd0], #current state 2 to next state
                  [0, 0, 0, 0, 0, 0, 0, 1-lmd0, 0, lmd0], #current state 3 to next state
                  [muM, 0, 0, 0, 1-muM, 0, 0, 0, 0, 0], #current state 4 to next state
                  [muM, 0, 0, 0, 0, 1-muM, 0, 0, 0, 0], #current state 5 to next state
                  [0, muM, 0, 0, 0, 0, 1-muM, 0, 0, 0], #current state 6 to next state
                  [0, 0, muM, 0, 0, 0, 0, 1-muM, 0, 0], #current state 7 to next state
                  [mu1, 0, 0, 0, 0, 0, 0, 0, 1-mu1, 0], #current state 8 to next state
                  [mu0, 0, 0, 0, 0, 0, 0, 0, 0, 1-mu0]]) #current state 9 to next state
tm = [a0_tm,a1_tm]
r_func = {0:2000,1:1500,2:1000,3:500,4:-maintenance_cost,5:-maintenance_cost,6:-maintenance_cost,7:-maintenance_cost,8:-3000,9:-2000}

class MachineEnv():
    def __init__(self,tm,r_func):
        self.action_space = [0,1]
        self.state = 0 #Random initialise the start state, assumes uniform distribution for initial state,random.randrange(10)
        self.state_seq = [] #initialise a list that records the actual states
        self.reward_func = r_func
        self.transition  = tm
        self.steps = 0
        self.done = False
    
    def step(self,action):
        transition_mat_action = self.transition[action]
        nxt_state = np.random.choice([i for i in range(10)],1,p=transition_mat_action[self.state])[0] #select nxt state based
        reward = self.reward_func[nxt_state]
        self.state = nxt_state #update state
        self.steps +=1
        
        if(((nxt_state == 0) and (self.steps > 20)) or (self.steps >= 50)):#condition for end of episode
            self.done = True
        
        return reward
    
    def reset(self):
        self.state = 0
        self.done= False
        

In [ ]:
# def compute_avg_return(environment, policy, steps,num_episodes=10):
#     total_return = 0.0
#     for _ in range(num_episodes):
#         environment.reset()
#         episode_return = 0.0 
#         for _ in range(steps):
#             action_step = policy[environment.state]
#             episode_return += environment.step(action_step)
#         total_return += episode_return   
#     avg_return = total_return / num_episodes
#     return avg_return# Evaluate the agent's policy once before training.

# Episodic Average Rewards

In [ ]:
def compute_avg_return(environment, policy,num_episodes):
    total_return = 0.0
    for _ in range(num_episodes):
        environment.reset()
        episode_return = 0.0 
        while(not machine.done):
            action_step = policy[environment.state]
            episode_return += environment.step(action_step)
        total_return += episode_return 
    avg_return = total_return / num_episodes
    return avg_return

In [ ]:
machine = MachineEnv(tm,r_func)

In [ ]:
import matplotlib.pyplot as plt

ep = 40
trained = []
iteration = [i for i in range(ep)]

for i in range(ep):
    print(f"Trial {i}")
    x2=compute_avg_return(machine, vi.policy, 30)
    trained.append(x2)
    
fig = plt.figure(figsize = (8,8))
plt.plot(iteration,trained)
plt.title("Average return over 1 episode")
plt.xlabel('Trial')
plt.ylabel('Average Return')

plt.show()
